In [1]:
from pyspark.sql import SparkSession, functions as F
from datetime import datetime

spark = SparkSession \
    .builder \
    .master("local[*]")\
    .appName("Exercicio Intro") \
    .getOrCreate()

In [2]:
schema = """id STRING, 
            tituloPincipal STRING, 
            tituloOriginal STRING, 
            anoLancamento INT,
            tempoMinutos INT,
            genero STRING,
            notaMedia DOUBLE,
            numeroVotos INT,
            generoArtista STRING,
            personagem STRING,
            nomeArtista STRING,
            anoNascimento INT,
            anoFalecimento INT,
            profissao STRING,
            titulosMaisConhecidos STRING
            """
df = spark.read.option("encoding", "UTF-8").csv('work', schema=schema, sep="|", header=True)
df.printSchema()
df.show(5)
print(df.count())  # 1045161 registros

root
 |-- id: string (nullable = true)
 |-- tituloPincipal: string (nullable = true)
 |-- tituloOriginal: string (nullable = true)
 |-- anoLancamento: integer (nullable = true)
 |-- tempoMinutos: integer (nullable = true)
 |-- genero: string (nullable = true)
 |-- notaMedia: double (nullable = true)
 |-- numeroVotos: integer (nullable = true)
 |-- generoArtista: string (nullable = true)
 |-- personagem: string (nullable = true)
 |-- nomeArtista: string (nullable = true)
 |-- anoNascimento: integer (nullable = true)
 |-- anoFalecimento: integer (nullable = true)
 |-- profissao: string (nullable = true)
 |-- titulosMaisConhecidos: string (nullable = true)

+---------+--------------+--------------+-------------+------------+-------+---------+-----------+-------------+--------------------+-----------------+-------------+--------------+------------+---------------------+
|       id|tituloPincipal|tituloOriginal|anoLancamento|tempoMinutos| genero|notaMedia|numeroVotos|generoArtista|         

In [3]:
# Drop de colunas indesejadas
colunas_indesejadas = ['generoArtista', 'anoNascimento',
                       'anoFalecimento', 'profissao', 'titulosMaisConhecidos']
df = df.drop(*colunas_indesejadas)

# Filtrando os gêneros desejados e renomeando a coluna com nome errado
df = df.withColumnRenamed('tituloPincipal', 'tituloPrincipal')
df = df.filter(F.col('genero').like('%Sci-Fi%') |
               F.col('genero').like('%Fantasy%'))
df.printSchema()
print(df.count())  # 61569 registros
print(df.select('id').distinct().count())  # Num de IDs distintos = 14303

root
 |-- id: string (nullable = true)
 |-- tituloPrincipal: string (nullable = true)
 |-- tituloOriginal: string (nullable = true)
 |-- anoLancamento: integer (nullable = true)
 |-- tempoMinutos: integer (nullable = true)
 |-- genero: string (nullable = true)
 |-- notaMedia: double (nullable = true)
 |-- numeroVotos: integer (nullable = true)
 |-- personagem: string (nullable = true)
 |-- nomeArtista: string (nullable = true)

61569
14303


In [4]:
# Eliminando registros duplicados e dados incompletos
df_novo = df.dropDuplicates()
df_novo = df_novo.dropna()
df.show()
print(df_novo.count())  # 57520

+---------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+--------------------+-------------------+
|       id|     tituloPrincipal|      tituloOriginal|anoLancamento|tempoMinutos|              genero|notaMedia|numeroVotos|          personagem|        nomeArtista|
+---------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+--------------------+-------------------+
|tt0000679|The Fairylogue an...|The Fairylogue an...|         1908|         120|   Adventure,Fantasy|      5.2|         66|The Wizard of Oz Man|      L. Frank Baum|
|tt0000679|The Fairylogue an...|The Fairylogue an...|         1908|         120|   Adventure,Fantasy|      5.2|         66|Nick Chopper - Ti...|   George E. Wilson|
|tt0000679|The Fairylogue an...|The Fairylogue an...|         1908|         120|   Adventure,Fantasy|      5.2|         66|His Majesty the S...|        Frank Burns|
|tt0000679

In [5]:
# Substituindo '\N' por valores nulos na coluna 'personagem'
# Esses valores não atrapalham a analise então não serão deletados
df_novo = df_novo.withColumn(
    'personagem', F.when(F.col('personagem') == '\\N', None)
    .otherwise(F.col('personagem')))

# Amostra de votos menor que 30 é considerada irrelevante (TCL)
df_novo = df_novo.filter(F.col("numeroVotos") >= 30)

In [6]:
# Verificando que não há inconsistências nos dados dos filmes
ids_distintos = df_novo.select('id').distinct().count()
print(ids_distintos)  # Resultado: 11081
ids_distintos = df_novo.select('id', 'tituloPrincipal', 'tituloOriginal',
                               'anoLancamento', 'tempoMinutos', 'genero',
                               'notaMedia', 'numeroVotos').distinct().count()
print(ids_distintos)  # Resultado: 11081

11081
11081


In [7]:
df_novo.orderBy('id').show()

+---------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+--------------------+-------------------+
|       id|     tituloPrincipal|      tituloOriginal|anoLancamento|tempoMinutos|              genero|notaMedia|numeroVotos|          personagem|        nomeArtista|
+---------+--------------------+--------------------+-------------+------------+--------------------+---------+-----------+--------------------+-------------------+
|tt0000679|The Fairylogue an...|The Fairylogue an...|         1908|         120|   Adventure,Fantasy|      5.2|         66|Tik-Tok - the Mac...|  Wallace Illington|
|tt0000679|The Fairylogue an...|The Fairylogue an...|         1908|         120|   Adventure,Fantasy|      5.2|         66|His Majesty the S...|        Frank Burns|
|tt0000679|The Fairylogue an...|The Fairylogue an...|         1908|         120|   Adventure,Fantasy|      5.2|         66|The Wizard of Oz Man|      L. Frank Baum|
|tt0000679

In [8]:
# Ultima checagagem dos dados
print(df_novo.count()) # 46835
df_novo.printSchema()
df_novo.describe().show()
df_novo.show()

46835
root
 |-- id: string (nullable = true)
 |-- tituloPrincipal: string (nullable = true)
 |-- tituloOriginal: string (nullable = true)
 |-- anoLancamento: integer (nullable = true)
 |-- tempoMinutos: integer (nullable = true)
 |-- genero: string (nullable = true)
 |-- notaMedia: double (nullable = true)
 |-- numeroVotos: integer (nullable = true)
 |-- personagem: string (nullable = true)
 |-- nomeArtista: string (nullable = true)

+-------+---------+---------------+--------------+-----------------+-----------------+--------------------+------------------+------------------+--------------------+--------------------+
|summary|       id|tituloPrincipal|tituloOriginal|    anoLancamento|     tempoMinutos|              genero|         notaMedia|       numeroVotos|          personagem|         nomeArtista|
+-------+---------+---------------+--------------+-----------------+-----------------+--------------------+------------------+------------------+--------------------+--------------------

In [9]:
teste = df_novo.groupBy('nomeArtista').count().orderBy(F.col('count').desc())
teste.show()

+--------------------+-----+
|         nomeArtista|count|
+--------------------+-----+
|     Lance Henriksen|   28|
|      John Carradine|   28|
|       Germán Valdés|   22|
|     Christopher Lee|   21|
|       Boris Karloff|   20|
|        Tom Sizemore|   19|
|        Nicolas Cage|   19|
|       Peter Cushing|   19|
|        Eric Roberts|   19|
|        David Warner|   18|
|      Lon Chaney Jr.|   17|
|Arnold Schwarzene...|   17|
|      Tadanobu Asano|   16|
|   Samuel L. Jackson|   16|
| Christopher Lambert|   16|
|    Malcolm McDowell|   16|
| Tony Chiu-Wai Leung|   15|
|    Michael Ironside|   15|
|         Johnny Depp|   15|
|        Keanu Reeves|   14|
+--------------------+-----+
only showing top 20 rows



In [10]:
teste.filter(F.col('nomeArtista').like('%Downey%')).show()

+-----------------+-----+
|      nomeArtista|count|
+-----------------+-----+
|Robert Downey Jr.|   11|
|    Thomas Downey|    3|
|Robert Downey Sr.|    1|
|    Javier Downey|    1|
+-----------------+-----+

